In [92]:
#!/usr/bin/env python2
import pandas as pd
import numpy as np
import copy
import seaborn as seab
from sklearn import metrics
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix, hstack

def add_feature(X, feature_to_add):
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('testtest.csv')
#print train_df.describe()


In [93]:
import warnings
warnings.filterwarnings('ignore')

print ('comments that are non toxic: '),
nontoxic=len(train_df[(train_df['toxic']==0) & (train_df['severe_toxic']==0) & (train_df['obscene']==0) & (train_df['threat']== 0) & (train_df['insult']==0) & (train_df['identity_hate']==0)])
print nontoxic 
print ('Percentage of comments that are non toxic:')
print (float(nontoxic)/len(train_df))*100

categories = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
train_comment = train_df.comment_text
test_comment = test_df.comment_text
#print test_comment.shape,train_comment.shape

data = train_df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
#corr = data.astype(float).corr()
#seab.heatmap(corr,linewidths=0.4,linecolor='white',annot=True)

vect = TfidfVectorizer(max_features=4000,stop_words='english')
train_vec = vect.fit_transform(train_comment)
test_vec = vect.transform(test_comment)
Zlist = {}
for clas in categories:
    Zlist[clas] = np.array(test_df[clas].tolist())

    

#print test_comment.shape,train_comment.shape
#print train_vec.shape
#Zlist = np.array(Zlist)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=12.0)
nbmodel = MultinomialNB()
#print Zlist

comments that are non toxic:  143346
Percentage of comments that are non toxic:
89.8321123512


In [94]:
resultantMatrix = []
#resultantMatrix.append(['Labels','toxic','severe_toxic','obscene','threat','insult','identity_hate'])
resultantMatrix.append(['Classification Technique','Average Accuracy'])
matindex = 0

#######################################################################

print "#logistic regression with binary relevance"


matindex +=1
resultantMatrix.append([])
resultantMatrix[matindex].append("Logistic Regression Binary Relevance")
for classes in categories:
    print 'Processing',classes
    y = train_df[classes]
    Z = Zlist[classes]
    logreg.fit(train_vec, y)
    y_pred_X = logreg.predict(test_vec)
    accuracy=accuracy_score(Z, y_pred_X)
    print 'Training accuracy :',accuracy
    resultantMatrix[matindex].append(accuracy)
    mett=metrics.classification_report(Z,y_pred_X)
    print mett
    #print logreg.predict_proba(test_vec)
    test_y_prob = logreg.predict_proba(test_vec)[:,1]
    #print test_y_prob

#logistic regression with binary relevance
Processing toxic
Training accuracy : 0.9238644534058582
              precision    recall  f1-score   support

           0       0.97      0.94      0.96     57888
           1       0.58      0.75      0.65      6090

   micro avg       0.92      0.92      0.92     63978
   macro avg       0.77      0.85      0.80     63978
weighted avg       0.93      0.92      0.93     63978

Processing severe_toxic
Training accuracy : 0.9923411172590578
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63611
           1       0.35      0.41      0.38       367

   micro avg       0.99      0.99      0.99     63978
   macro avg       0.68      0.70      0.69     63978
weighted avg       0.99      0.99      0.99     63978

Processing obscene
Training accuracy : 0.9626121479258495
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     60287
           1     

In [95]:
#dummy = []
#######################################################################
print "#Naive Bayes with binary relevance"
matindex +=1
resultantMatrix.append([])
resultantMatrix[matindex].append("MultinomialNB Binary Relevance")
for classes in categories:
    print 'Processing',classes
    y = train_df[classes]
    Z = Zlist[classes]
    #logreg.fit(train_vec, y)
    model = MultinomialNB()
    model.fit(train_vec, y)  
    #y_pred_X = logreg.predict(test_vec)
    y_pred_X= model.predict(test_vec)
    #dummy.append(y_pred_X[0])
    accuracy=accuracy_score(Z, y_pred_X)
    print 'Training accuracy :',accuracy
    resultantMatrix[matindex].append(accuracy)
    mett=metrics.classification_report(Z,y_pred_X)
    print mett
    #test_y_prob = model.predict_proba(test_vec)[:,1]
    #dummy.append(test_y_prob)
#print dummy

#Naive Bayes with binary relevance
Processing toxic
Training accuracy : 0.9346025196161181
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     57888
           1       0.69      0.56      0.62      6090

   micro avg       0.93      0.93      0.93     63978
   macro avg       0.82      0.77      0.79     63978
weighted avg       0.93      0.93      0.93     63978

Processing severe_toxic
Training accuracy : 0.9937634811966614
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63611
           1       0.45      0.37      0.41       367

   micro avg       0.99      0.99      0.99     63978
   macro avg       0.72      0.69      0.70     63978
weighted avg       0.99      0.99      0.99     63978

Processing obscene
Training accuracy : 0.9640188814905124
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     60287
           1       0.78  

In [96]:
#######################################################################
print "# Logistic regression with classifier chain"

matindex +=1
resultantMatrix.append([])
resultantMatrix[matindex].append("Logistic Regression Classifier Chain")
test_vec_chain = copy.deepcopy(test_vec)
train_vec_chain = copy.deepcopy(train_vec)
for classes in categories:
    print 'Processing',classes
    y = train_df[classes]
    Z = Zlist[classes]
    #model = MultinomialNB()
    #logreg.fit(train_vec, y)
    logreg.fit(train_vec_chain, y)  
    y_pred_X = logreg.predict(test_vec_chain)
    #y_pred_X= model.predict(test_vec_chain)
    accuracy=accuracy_score(Z, y_pred_X)
    print 'Training accuracy :',accuracy
    resultantMatrix[matindex].append(accuracy)
    mett=metrics.classification_report(Z,y_pred_X)
    print mett    
    #test_y=logreg.predict(test_vec)
    #test_y_prob = model.predict_proba(test_vec)[:,1]
    train_vec_chain = add_feature(train_vec_chain, y)
    print('Shape of train_vec is now {}'.format(train_vec_chain.shape))
    # chain current label predictions to test_train_vec
    test_vec_chain = add_feature(test_vec_chain, y_pred_X)
    print('Shape of test_train_vec is now {}'.format(test_vec_chain.shape))

# Logistic regression with classifier chain
Processing toxic
Training accuracy : 0.9238644534058582
              precision    recall  f1-score   support

           0       0.97      0.94      0.96     57888
           1       0.58      0.75      0.65      6090

   micro avg       0.92      0.92      0.92     63978
   macro avg       0.77      0.85      0.80     63978
weighted avg       0.93      0.92      0.93     63978

Shape of train_vec is now (159571, 4001)
Shape of test_train_vec is now (63978, 4001)
Processing severe_toxic
Training accuracy : 0.9923411172590578
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     63611
           1       0.34      0.35      0.34       367

   micro avg       0.99      0.99      0.99     63978
   macro avg       0.67      0.67      0.67     63978
weighted avg       0.99      0.99      0.99     63978

Shape of train_vec is now (159571, 4002)
Shape of test_train_vec is now (63978, 4002)
Processing 

In [97]:
#######################################################################
print "#Naive Bayes with classifier chain"
matindex +=1
resultantMatrix.append([])
resultantMatrix[matindex].append("MultinomialNB Classifier Chain")
test_vec_chain = copy.deepcopy(test_vec)
train_vec_chain = copy.deepcopy(train_vec)
for classes in categories:
    print 'Processing',classes
    y = train_df[classes]
    Z = Zlist[classes]
    nbmodel.fit(train_vec_chain, y)  
    y_pred_X= nbmodel.predict(test_vec_chain)
    accuracy=accuracy_score(Z, y_pred_X)
    print 'Training accuracy :',accuracy
    resultantMatrix[matindex].append(accuracy)
    mett=metrics.classification_report(Z,y_pred_X)
    print mett    
    train_vec_chain = add_feature(train_vec_chain, y)
    print('Shape of train_vec is now {}'.format(train_vec_chain.shape))
    # chain current label predictions to test_train_vec
    test_vec_chain = add_feature(test_vec_chain, y_pred_X)
    print('Shape of test_train_vec is now {}'.format(test_vec_chain.shape))
#######################################################################    
import csv
with open("graph.csv", 'w') as writeFile:
    writer = csv.DictWriter(writeFile, fieldnames=resultantMatrix[0])
    writer = csv.writer(writeFile)
    writer.writerow(resultantMatrix[0])
    for row in range(1,len(resultantMatrix)):
        roww = []
        roww.append(resultantMatrix[row][0])
        val = 0.0
        for j in range(1,len(resultantMatrix[row])):
            val += resultantMatrix[row][j]
        roww.append(val / (len(resultantMatrix[row]) - 1))
        writer.writerow(roww)

#Naive Bayes with classifier chain
Processing toxic
Training accuracy : 0.9346025196161181
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     57888
           1       0.69      0.56      0.62      6090

   micro avg       0.93      0.93      0.93     63978
   macro avg       0.82      0.77      0.79     63978
weighted avg       0.93      0.93      0.93     63978

Shape of train_vec is now (159571, 4001)
Shape of test_train_vec is now (63978, 4001)
Processing severe_toxic
Training accuracy : 0.9841039107193098
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     63611
           1       0.22      0.70      0.34       367

   micro avg       0.98      0.98      0.98     63978
   macro avg       0.61      0.84      0.66     63978
weighted avg       0.99      0.98      0.99     63978

Shape of train_vec is now (159571, 4002)
Shape of test_train_vec is now (63978, 4002)
Processing obscene
T

In [98]:
#print resultantMatrix


In [105]:
A = np.array([np.array([1,2,3]),np.array([7,8,9])])
B = np.array([np.array([4,5,6]),np.array([10,11,12])])

In [110]:
for i in range (len(A)):
    print np.concatenate((A[i],B[i]),axis=0)

[1 2 3 4 5 6]
[ 7  8  9 10 11 12]


In [122]:
XX = [[1],[2]]
ZZ = [4,5]
print XX + ZZ

[[1], [2], 4, 5]


In [134]:
print add_feature(XX,ZZ).toarray()

[[1 4]
 [2 5]]


In [132]:
print add_feature(XX,ZZ)[1,1]

5
